In [1]:
import csv
import os
from collections import Counter
from collections import defaultdict
from string import punctuation

import enchant
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams

# Lexicon

In [2]:
def remove_accent_marks(w):
    r = w
    r = r.replace('á', 'a')
    r = r.replace('é', 'e')
    r = r.replace('í', 'i')
    r = r.replace('ó', 'o')
    r = r.replace('ú', 'u')
    return r

In [3]:
lexicon = {}
with open('lexicon/es_lexicon.csv') as f:
    reader = csv.reader(
        f,
        delimiter=' ',
    )
    for row in reader:
        w = remove_accent_marks(row[0])
        lexicon[w] = []
        for i in range(1, len(row[1:]), 2):
            entry = {}
            entry['lemma'] = remove_accent_marks(row[i].lower())
            entry['eagle'] = remove_accent_marks(row[i+1].lower())
            lexicon[w].append(entry)

In [4]:
len(lexicon)

500949

In [5]:
def is_vowel(c):
    return c in 'aeiouAEIOUáéíóúÁÉÍÓÚüÜ'

def next_level(w):
    result = [w]
    if is_vowel(w[0]):
        result.append('h' + w)
    if "q'" in w:
        result.append(w.replace("q'", 'que'))
    for i in range(len(w)):
        if w[i] == 'v':
            result.append(w[:i] + 'b' + w[i+1:])
        elif w[i] == 'b':
            result.append(w[:i] + 'v' + w[i+1:])
            result.append(w[:i] + 'd' + w[i+1:])
        elif w[i] == 'd':
            result.append(w[:i] + 'b' + w[i+1:])
        elif w[i] == 'c' and i < len(w) - 1 and w[i+1] in 'ei':
            result.append(w[:i] + 's' + w[i+1:])
        elif w[i] == 's' and i < len(w) - 1 and w[i+1] in 'ei':
            result.append(w[:i] + 'c' + w[i+1:])
    return result

def combinations_aux(visited, result):
    while visited:
        w = visited[0]
        visited = visited[1:]
        if w not in result:
            result.append(w)
            combs = next_level(w)
            visited.extend(combs)        
    return result

def combinations(w):
    return combinations_aux([w], [])

In [6]:
es = enchant.Dict('es_ES')

# Analysis

In [7]:
os.listdir('data')

['ana.csv', 'dago.csv', 'rafa.csv', 'final.csv', 'yadira.csv']

In [8]:
d = {}
for filename in os.listdir('data'):
    with open('data/' + filename) as f:
        reader = csv.reader(
            f,
            delimiter=',',
            quotechar='"'
        )
        next(reader)
        for row in reader:
            d[row[0]] = {
                'date': row[1],
                'name': row[2],
                'gender': row[3],
                'age': row[4],
                'city': row[5],
                'drawings': row[6],
                'title': row[7],
                'description': row[8],
            }

In [9]:
len(d)

826

## Gender

In [10]:
blank = 0
male = 0
female = 0
for k in d:
    gender = d[k]['gender']
    if gender == 'M':
        male += 1
    elif gender == 'F':
        female += 1
    else:
        blank += 1

In [11]:
blank

449

In [12]:
male

108

In [13]:
female

269

## Cities

In [14]:
cities = []
for k in d:
    city = d[k]['city']
    cities.append(city)

In [15]:
c = Counter(cities)

In [16]:
# c

In [17]:
c.most_common(10)

[('', 582),
 ('Bogotá', 155),
 ('Bogotá D.C.', 9),
 ('Neiva', 7),
 ('Fusagasugá', 6),
 ('Bogotá D.C', 5),
 ('Soacha', 5),
 ('Bucaramanga', 4),
 ('Cali', 3),
 ('Medellín', 3)]

## Ages

In [18]:
ages = []
for k in d:
    age = d[k]['age']
    ages.append(age)

In [19]:
a = Counter(ages)

In [20]:
# a

In [21]:
a.most_common(11)

[('', 575),
 ('14', 16),
 ('16', 13),
 ('10', 12),
 ('19', 12),
 ('20', 11),
 ('11', 11),
 ('13', 11),
 ('12', 10),
 ('21', 9),
 ('17', 9)]

In [22]:
children = 0
teenagers = 0
young_adults = 0
adults = 0
for k in d:
    try:
        if d[k]['age']:
            age = int(d[k]['age'])
            if 0 <= age <= 12:
                children += 1
            elif 13 <= age <= 18:
                teenagers += 1
            elif 19 <= age <= 25:
                young_adults += 1
            else:
                adults += 1
    except: # '16, 19'
        teenagers += 1

In [23]:
children

51

In [24]:
teenagers

65

In [25]:
young_adults

54

In [26]:
adults

81

## Drawings

In [27]:
drawing_words = []
for k in d:
    drawings = d[k]['drawings'].lower().replace(';', ',')
    words = [x.strip() for x in drawings.split(',')]
    drawing_words.extend(words)

In [28]:
dw = Counter(drawing_words)

In [29]:
# dw

In [30]:
dw.most_common(11)

[('', 715),
 ('corazón', 84),
 ('cara feliz', 28),
 ('niño', 11),
 ('carita feliz', 11),
 ('nube', 7),
 ('cara sonriente', 5),
 ('perro', 4),
 ('sol', 3),
 ('ojos', 2),
 ('flor', 2)]

## Words

In [31]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
def normalize(text):
    text = text.lower()
    text = remove_accent_marks(text)
    for p in punctuation:
        text = text.replace(p, '')
    return text

In [33]:
def total_text(r):
    return normalize(r['title'] + ' ' + r['description'])

In [34]:
total_words = []
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    total_words.extend(words)

In [35]:
len(total_words)

45543

In [36]:
total_words_freq = Counter(total_words)

In [37]:
total_words_freq.most_common(10)

[('de', 1996),
 ('y', 1954),
 ('la', 1684),
 ('que', 1365),
 ('en', 1224),
 ('a', 1193),
 ('el', 1032),
 ('un', 812),
 ('con', 692),
 ('mi', 668)]

In [38]:
stop_words = [remove_accent_marks(w) for w in stopwords.words('spanish')]

In [39]:
total_nonstop_words = [w for w in total_words if w not in stop_words]

In [40]:
len(total_nonstop_words)

21983

In [41]:
unique_nonstop_words = set(total_nonstop_words)

In [42]:
len(unique_nonstop_words)

6675

## Spelling

In [43]:
correct_spelling = {}

for w in total_words_freq:
    if w not in correct_spelling:
        if w in lexicon:
            correct_spelling[w] = w
        else:
            combs = combinations(w)
            freqs = [(c, total_words_freq[c]) for c in combs if c in total_words_freq and c in lexicon]
            if freqs:
                freqs.sort(key=lambda x: x[1], reverse=True)
                correct_spelling[w] = freqs[0][0]
            else:
                freqs = [(c, total_words_freq[c]) for c in es.suggest(w) if c in total_words_freq]
                if freqs:
                    freqs.sort(key=lambda x: x[1], reverse=True)
                    correct_spelling[w] = freqs[0][0]
                else:
                    correct_spelling[w] = w

In [44]:
len(correct_spelling)

6858

In [45]:
len(total_words_freq)

6858

In [46]:
spelling_map = defaultdict(list)
for key in correct_spelling:
    v = correct_spelling[key]
    spelling_map[v].append((key, total_words_freq[key]))
spelling_map = dict(spelling_map)

In [47]:
len(spelling_map)

6306

In [48]:
correct_spelling['aser']

'hacer'

In [49]:
spelling_map['hacer']

[('aser', 3), ('acer', 1), ('hacer', 71), ('haser', 1)]

In [50]:
mispellings = [w for w in spelling_map if w not in stop_words and len(spelling_map[w]) > 1]
mispellings.sort()

In [51]:
mispellings[:10]

['abandonados',
 'abrasamos',
 'abuelas',
 'abuelo',
 'abuelos',
 'abusar',
 'acabo',
 'aceptar',
 'aceptaron',
 'adoptar']

In [52]:
spelling_map['abandonados']

[('habandonados', 1), ('abandonados', 3)]

## n-grams

In [53]:
def all_stopwords(ngrm):
    return all(map(lambda x: x in stop_words, ngrm))

In [54]:
def my_ngrams(tokens):
    ngrms = []
    for i in range(1, 3 + 1):
        ngrms_aux = [ngrm for ngrm in ngrams(tokens, i) if not all_stopwords(ngrm)]
        ngrms.extend(ngrms_aux)
    return ngrms

In [55]:
total_unigrams = []
total_bigrams = []
total_trigrams = []
for k in d:
    clean_title_words = [correct_spelling[w] for w in word_tokenize(normalize(d[k]['title']))]
    clean_description_words = [correct_spelling[w] for w in word_tokenize(normalize(d[k]['description']))]
    title_ngrams = my_ngrams(clean_title_words)
    description_ngrams = my_ngrams(clean_description_words)
    text_ngrams = title_ngrams + description_ngrams
    total_unigrams.extend(ngrm for ngrm in text_ngrams if len(ngrm) == 1)
    total_bigrams.extend(ngrm for ngrm in text_ngrams if len(ngrm) == 2)
    total_trigrams.extend(ngrm for ngrm in text_ngrams if len(ngrm) == 3)

In [56]:
tuni = Counter(total_unigrams)

In [57]:
tuni.most_common(100)

[(('paz',), 668),
 (('dia',), 240),
 (('familia',), 153),
 (('ser',), 135),
 (('personas',), 132),
 (('vida',), 118),
 (('amor',), 102),
 (('cada',), 90),
 (('años',), 88),
 (('niños',), 88),
 (('hecho',), 88),
 (('mejor',), 87),
 (('despues',), 82),
 (('perdon',), 82),
 (('colegio',), 82),
 (('asi',), 79),
 (('solo',), 79),
 (('hacer',), 76),
 (('tiempo',), 73),
 (('siempre',), 70),
 (('casa',), 69),
 (('año',), 65),
 (('hace',), 65),
 (('persona',), 65),
 (('amigos',), 60),
 (('mama',), 58),
 (('demas',), 53),
 (('hogar',), 53),
 (('trabajo',), 52),
 (('colombiano',), 51),
 (('dos',), 50),
 (('compañeros',), 50),
 (('mundo',), 50),
 (('mal',), 49),
 (('hoy',), 49),
 (('dias',), 48),
 (('respeto',), 48),
 (('ahora',), 48),
 (('cosas',), 48),
 (('pais',), 48),
 (('violencia',), 48),
 (('perdonar',), 46),
 (('corazon',), 46),
 (('bien',), 45),
 (('niño',), 45),
 (('social',), 45),
 (('calle',), 44),
 (('parte',), 44),
 (('cuenta',), 44),
 (('ayuda',), 43),
 (('ayudar',), 43),
 (('tener'

In [58]:
tbi = Counter(total_bigrams)

In [59]:
tbi.most_common(100)

[(('la', 'paz'), 355),
 (('de', 'paz'), 125),
 (('un', 'dia'), 108),
 (('paz', 'es'), 68),
 (('las', 'personas'), 58),
 (('mi', 'familia'), 56),
 (('paz', 'en'), 52),
 (('hecho', 'de'), 52),
 (('en', 'paz'), 50),
 (('despues', 'de'), 48),
 (('la', 'vida'), 45),
 (('paz', 'y'), 44),
 (('los', 'niños'), 42),
 (('mi', 'mama'), 40),
 (('la', 'familia'), 38),
 (('los', 'demas'), 38),
 (('la', 'calle'), 38),
 (('personas', 'que'), 34),
 (('el', 'colegio'), 33),
 (('grupo', 'de'), 31),
 (('asi', 'que'), 31),
 (('un', 'grupo'), 31),
 (('mi', 'hermano'), 30),
 (('el', 'amor'), 30),
 (('parte', 'de'), 30),
 (('el', 'perdon'), 28),
 (('la', 'violencia'), 27),
 (('le', 'dije'), 27),
 (('mis', 'compañeros'), 26),
 (('el', 'año'), 26),
 (('mi', 'padre'), 25),
 (('mi', 'hermana'), 25),
 (('un', 'hecho'), 25),
 (('ayudar', 'a'), 24),
 (('ese', 'dia'), 24),
 (('el', 'dia'), 24),
 (('acto', 'de'), 23),
 (('paz', 'con'), 23),
 (('ayuda', 'a'), 23),
 (('medio', 'de'), 23),
 (('familia', 'y'), 22),
 (('mi'

In [60]:
ttri = Counter(total_trigrams)

In [61]:
ttri.most_common(100)

[(('la', 'paz', 'es'), 50),
 (('hecho', 'de', 'paz'), 41),
 (('a', 'la', 'paz'), 29),
 (('en', 'el', 'colegio'), 28),
 (('un', 'grupo', 'de'), 25),
 (('la', 'paz', 'en'), 23),
 (('a', 'los', 'demas'), 22),
 (('para', 'la', 'paz'), 21),
 (('las', 'personas', 'que'), 20),
 (('de', 'la', 'paz'), 17),
 (('un', 'hecho', 'de'), 17),
 (('de', 'la', 'calle'), 17),
 (('en', 'la', 'calle'), 17),
 (('que', 'la', 'paz'), 17),
 (('en', 'el', 'año'), 16),
 (('a', 'las', 'personas'), 16),
 (('mi', 'hecho', 'de'), 15),
 (('la', 'paz', 'se'), 15),
 (('me', 'di', 'cuenta'), 14),
 (('la', 'oportunidad', 'de'), 14),
 (('la', 'paz', 'y'), 14),
 (('todos', 'los', 'dias'), 14),
 (('un', 'dia', 'en'), 13),
 (('por', 'la', 'paz'), 13),
 (('con', 'mi', 'mama'), 13),
 (('con', 'mi', 'hermana'), 13),
 (('con', 'mi', 'hermano'), 13),
 (('a', 'traves', 'de'), 13),
 (('por', 'medio', 'de'), 13),
 (('de', 'paz', 'es'), 12),
 (('con', 'mi', 'familia'), 12),
 (('a', 'los', 'niños'), 12),
 (('a', 'pesar', 'de'), 12),
 (

## Animales

In [62]:
animales = ['animal', 'animalito', 'animales', 'animalitos',
            'perro', 'perra', 'perros', 'perras', 'perrito', 'perrita', 'perritos', 'perritas',
            'gato', 'gata', 'gatos', 'gatas', 'gatito', 'gatita', 'gatitos', 'gatitas',
            'mascota', 'ovejita', 'pez']

In [63]:
# animales = ['caballo', 'yegua', 'caballito', 'burro', 'burrito', 'oveja',
#             'cabra', 'vaquita', 'ovejita', 'vaca', 'cordero', 'corderito',
#             'pez', 'pececito', 'cerdo', 'cerdito', 'puerco', 'gallina', 'pollo', 'gallo',
#             'pesesito', 'pes']

In [64]:
total_animales = []
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    for a in words:
        if a in animales:
            total_animales.append(a)

In [65]:
len(total_animales)

136

In [66]:
ta = Counter(total_animales)

In [67]:
ta

Counter({'animal': 13,
         'animales': 26,
         'animalito': 1,
         'animalitos': 1,
         'gata': 3,
         'gatito': 3,
         'gato': 13,
         'gatos': 6,
         'mascota': 6,
         'ovejita': 1,
         'perra': 4,
         'perrita': 6,
         'perrito': 22,
         'perritos': 5,
         'perro': 13,
         'perros': 11,
         'pez': 2})

In [68]:
total_animales_m = []
total_animales_f = []
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    for a in words:
        if a in animales:
            if d[k]['gender'] == 'M':
                total_animales_m.append(a)
            elif d[k]['gender'] == 'F':
                total_animales_f.append(a)

In [69]:
len(total_animales_m)

20

In [70]:
len(total_animales_f)

52

In [71]:
Counter(total_animales_m).most_common()

[('perrito', 5),
 ('perro', 4),
 ('perritos', 3),
 ('gato', 3),
 ('animales', 2),
 ('mascota', 2),
 ('perros', 1)]

In [72]:
Counter(total_animales_f).most_common()

[('animales', 15),
 ('animal', 8),
 ('gato', 7),
 ('perros', 6),
 ('perro', 3),
 ('perrito', 3),
 ('perra', 2),
 ('perrita', 2),
 ('animalitos', 1),
 ('gatos', 1),
 ('animalito', 1),
 ('gata', 1),
 ('gatito', 1),
 ('mascota', 1)]

In [73]:
total_animales_age = []
total_animales_noage = []
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    for a in words:
        if a in animales:
            if d[k]['age']:
                total_animales_age.append(a)
            else:
                total_animales_noage.append(a)

In [74]:
len(total_animales_age)

51

In [75]:
len(total_animales_noage)

85

In [76]:
total_animales_ch = []
total_animales_te = []
total_animales_ya = []
total_animales_ad = []
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    for a in words:
        if a in animales:
            try:
                if d[k]['age']:
                    age = int(d[k]['age'])
                    if 0 <= age <= 12:
                        total_animales_ch.append(a)
                    elif 13 <= age <= 18:
                        total_animales_te.append(a)
                    elif 19 <= age <= 25:
                        total_animales_ya.append(a)
                    else:
                        total_animales_ad.append(a)
            except: # '16, 19'
                total_animales_te.append(a)

In [77]:
len(total_animales_ch)

27

In [78]:
Counter(total_animales_ch).most_common()

[('gato', 8),
 ('perrito', 5),
 ('perro', 4),
 ('animal', 3),
 ('animales', 3),
 ('mascota', 2),
 ('animalito', 1),
 ('perritos', 1)]

In [79]:
len(total_animales_te)

12

In [80]:
Counter(total_animales_te).most_common()

[('perrito', 3),
 ('animal', 3),
 ('animales', 3),
 ('perros', 1),
 ('mascota', 1),
 ('gato', 1)]

In [81]:
len(total_animales_ya)

6

In [82]:
Counter(total_animales_ya).most_common()

[('perra', 2), ('animales', 2), ('perrita', 1), ('animal', 1)]

In [83]:
len(total_animales_ad)

6

In [84]:
Counter(total_animales_ad).most_common()

[('animales', 4), ('perros', 1), ('gata', 1)]

In [85]:
total_cards_with_animales = 0
for k in d:
    text = total_text(d[k])
    words = [w for w in word_tokenize(text) if w not in punctuation]
    for a in words:
        if a in animales:
            total_cards_with_animales += 1
            break

In [86]:
total_cards_with_animales

63